## Use MLflow to Experiment a Keras Network Model Binary Classification for Movie Reviews

This notebook is based on 
* Jules Damji's ([@2twitme](https://twitter.com/2twitme)) blog [How to Use MLflow to Experiment a Keras Network Model: Binary Classification for Movie Reviews](https://databricks.com/blog/2018/08/23/how-to-use-mlflow-to-experiment-a-keras-network-model-binary-classification-for-movie-reviews.html) 
* [jsd-mlflow-examples](https://github.com/dmatrix/jsd-mlflow-examples)
* Francois Cholllet's book: [_Deep Learning with Python_](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff)


Pre-requisites:
* Refer to [MLflow Quick Start: Model Training and Logging](https://docs.databricks.com/spark/latest/mllib/mlflow.html) to setup an MLflow tracking server on a Linux instance.

In [2]:
import mlflow

# Set global variable for output_dir
output_dir = ""

AttributeError Traceback (most recent call last)
 <command-1440947> in <module> () 
 3 
 4 # Set experiment ID 
 ----> 5 mlflow . set_experiment_id ( 11 ) 
 6 
 7 # Set global variable for output_dir 

 AttributeError : 'module' object has no attribute 'set_experiment_id'

In [3]:
from keras.datasets import imdb
import numpy as np

"""
Part of this module is derived and borrowed heavily from Francois Chollet's book Deep Learning Python. Original code
can be found at :https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/3.5-classifying-movie-reviews.ipynb

We'll be working with "IMDB dataset", a set of 50,000 highly-polarized reviews from the Internet Movie Database. They are split into 
25,000 reviews for training and 25,000 reviews for testing, each set consisting in 50% negative and 50% positive reviews.
Why do we have these two separate training and test sets? You should never test a machine learning model on the same data 
that you used to train it! Just because a model performs well on its training data doesn't mean that it will perform well 
on data it has never seen, and what you actually care about is your model's performance on new data (since you already 
know the labels of your training data -- obviously you don't need your model to predict those). For instance, it is 
possible that your model could end up merely memorizing a mapping between your training samples and their targets -- 
which would be completely useless for the task of predicting targets for data never seen before. We will go over this 
point in much more detail in the next chapter.

Just like the MNIST dataset, the IMDB dataset comes packaged with Keras. It has already been preprocessed: the reviews 
(sequences of words) have been turned into sequences of integers, where each integer stands for a specific word in a dictionary.

The following code will load the dataset (when you run it for the first time, about 80MB of data will be downloaded to your 
machine)
"""

class KIMDB_Data_Utils():

    def __init__(self):
        return

    def fetch_imdb_data(self, num_words=10000):
        """
        :param num_words: This arguments meants that we want to keep the top 10,000 most frequently occuring words in the training data. Rare words will be discarded
        :return: The variables train_data and test_data are lists of reviews, each review being a list of word indices (encoding a sequence of words).  train_labels and test_labels are lists of 0s and 1s, where 0 stands for "negative" \
        and 1 stands for "positive":
        """
        (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=num_words)

        return (train_data, train_labels), (test_data, test_labels)

    def decode_review(self, train_data, index=0):
        """
        Return a decoded review
        :param index: is index into mapping of words into the integer index
        :return: a string matching the review
        """
        # word_index is a dictionary mapping words to an integer index
        word_index = imdb.get_word_index()
        # We reverse it, mapping integer indices to words
        reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
        # We decode the review; note that our indices were offset by 3
        # because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
        decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[index]])

        return decoded_review

    def prepare_vectorized_sequences(self, sequences, dimension=10000):
        """
        We cannot feed lists of integers into a neural network. We have to turn our lists into tensors. One way is to convert the sequence
        into tensors using Numpy. Also, we are going to use one-hot-encode our lists into vectors of 0s and 1s. That is, for instance turning the sequence
        [3, 5] into a 10,000-dimensional vector that would be all-zeros except for indices 3 and 5, which would be ones. Then we could use as first layer in our
        network a Dense layer, capable of handling floating point vector data.
        :param sequences: this is the sequence we want to convert
        :param dimension: size of the sequence
        :return: list of one-hot-encoded vector []
        """
        # Create an all-zero matrix of shape (len(sequences), dimension)
        results = np.zeros((len(sequences), dimension))
        for i, sequence in enumerate(sequences):
            results[i, sequence] = 1.  # set specific indices of results[i] to 1s

        return results

    def prepare_vectorized_labels(self, labels):
        """
        labels are scalars so we can just use numpy as arrays of type float
        :param labels: label data
        :return: numpy array
        """
        return np.asarray(labels).astype('float32')
#
# Test the functions
#
if __name__ == '__main__':
    # create a class handle
    kdata_cls = KIMDB_Data_Utils()
    (train_data, train_labels), (test_data, test_labels) = kdata_cls.fetch_imdb_data(num_words=10000)
    print(train_data[0])
    print(len(train_data))
    decoded = kdata_cls.decode_review(train_data)
    print(decoded)
    x_train = kdata_cls.prepare_vectorized_sequences(train_data)
    x_test = kdata_cls.prepare_vectorized_sequences(test_data)
    print(x_train[0])
    print(x_test[0])
    y_train = kdata_cls.prepare_vectorized_labels(train_labels)
    y_test = kdata_cls.prepare_vectorized_labels(test_labels)
    print(y_train)
    print(y_test)

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
25000
? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all
[0. 1. 1. ... 0. 0. 0.]
[0. 1. 1. ... 0. 0. 0.]
[1. 0. 0. ... 0. 1. 0.]
[0. 1. 1. ... 0. 0. 0.]

In [4]:
from keras import models
from keras import layers

class KModel():

    def __init__(self):
        return

    def build_basic_model(self):

        """
        Build the base line model with one input layer, one hidden layer, and one output layer, with
        16, 16, and 1 output neurons. Default activation functions for input and hidden layer are relu
        and sigmoid respectively
        :return: a Keras network model
        """

        base_model = models.Sequential()
        base_model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
        base_model.add(layers.Dense(16, activation='relu'))
        base_model.add(layers.Dense(1, activation='sigmoid'))

        return base_model

    def build_experimental_model(self, hidden_layers=1, output=16, activation='relu'):

        exp_model = models.Sequential()
        # add the input layers
        exp_model.add(layers.Dense(output, activation=activation, input_shape=(10000,)))
        # add hidden layers
        for i in range(0, hidden_layers):
            exp_model.add(layers.Dense(output, activation=activation))
        # add output layer
        exp_model.add(layers.Dense(1, activation='sigmoid'))

        return exp_model

if __name__ == '__main__':

    mmaker = KModel()
    model = mmaker.build_basic_model()
    model.summary()

    custom_model = mmaker.build_experimental_model(3, 32, 'tanh')
    custom_model.summary()

_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
dense_1 (Dense) (None, 16) 160016 
_________________________________________________________________
dense_2 (Dense) (None, 16) 272 
_________________________________________________________________
dense_3 (Dense) (None, 1) 17 
=================================================================
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
dense_4 (Dense) (None, 32) 320032 
_________________________________________________________________
dense_5 (Dense) (None, 32) 1056 
_________________________________________________________________
dense_6 (Dense) (None, 32) 1056 
_________________________________________________________________
dense_7 (Dense) (None, 32) 1056 
_________________________________________________________________
dense_8 (Dense) (None, 1) 33 
=================================================================
Total params: 323,233
Trainable params: 323,233
Non-trainable params: 0
_________________________________________________________________

In [5]:
import matplotlib.pyplot as plt

class KPlot():
    
    def __init__(self):
        return

    def plot_loss_graph(self,history, title):
        """
        Generate a matplotlib graph for the loss and accuracy metrics
        :param histroy:
        :return: instance of a graph
        """

        acc = history.history['binary_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']

        epochs = range(1, len(acc) + 1)

        fig, ax = plt.subplots()
        ax.plot(epochs, loss, 'bo')

        # "bo" is for "blue dot"
        plt.plot(epochs, loss, 'bo', label='Training loss')
        # b is for "solid blue line"
        plt.plot(epochs, val_loss, 'b', label='Validation loss')
        plt.title(title)
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        return fig

    def plot_accuracy_graph(elf, history, title):

        plt.clf()
        acc = history.history['binary_accuracy']
        val_acc = history.history['val_binary_accuracy']

        epochs = range(1, len(acc) + 1)

        fig, ax = plt.subplots()
        ax.plot(epochs, acc, 'bo')

        plt.plot(epochs, acc, 'bo', label='Training acc')
        plt.plot(epochs, val_acc, 'b', label='Validation acc')
        plt.title(title)
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        return fig

In [6]:
# Create Params dictionary
class Params(object):
	def __init__(self, hidden_layers, epochs, output, loss):
		self.hidden_layers = hidden_layers
		self.epochs = epochs
		self.output = output
		self.loss = loss

# Configure args
#args = Params(1, 20, 16, "binary_crossentropy")
#args = Params(3, 30, 16, "binary_crossentropy")
#args = Params(3, 20, 32, "mse")

In [7]:
import os
import sys
import mlflow
import mlflow.keras
import tensorflow as tf
import tempfile

from keras import optimizers
from keras import metrics

class KTrain():

    def __init__(self):
        return

    def compile_and_fit_model(self, model, x_train, y_train, epochs=20, batch_size=512, loss='binary_crossentropy',
                              optimizer='rmsprop', lr=0.0001, metrics=metrics.binary_accuracy,
                              verbose=1, save_model=0):
        #
        # generate validation data and training data
        #
        x_val = x_train[:10000]
        partial_x_train = x_train[10000:]

        y_val = y_train[:10000]
        partail_y_train = y_train[10000:]

        if optimizer == 'rmsprop':
            opt = optimizers.RMSprop(lr=lr)
        model.compile(optimizer=opt,
                      loss=loss,
                      metrics=[metrics])
        #
        # fit the model: use part of the training data and use validation for unseen data
        #
        history = model.fit(partial_x_train,
                            partail_y_train,
                            epochs=epochs,
                            batch_size=batch_size,
                            verbose=verbose,
                            validation_data=(x_val, y_val))

        if save_model:
            pathdir = "keras_models/" + run_uuid
            model_dir = self.get_directory_path(pathdir)
            self.keras_save_model(model, model_dir)

        return history

    def keras_save_model(self, model, model_dir='/tmp'):
        """
        Convert Keras estimator to TensorFlow
        :type model_dir: object
        """
        print("Keras model saved locally at %s" % model_dir)
        mlflow.keras.save_model(model, model_dir)

    def evaulate_model(self,model, x_test, y_test):
        """
        Evaulate the model with unseen and untrained data
        :param model:
        :return: results of probability
        """

        return model.evaluate(x_test, y_test)

    def get_binary_loss(self, hist):
        loss = hist.history['loss']
        loss_val = loss[len(loss) - 1]
        return loss_val

    def get_binary_acc(self, hist):
        acc = hist.history['binary_accuracy']
        acc_value = acc[len(acc) - 1]

        return acc_value

    def get_validation_loss(self, hist):
        val_loss = hist.history['val_loss']
        val_loss_value = val_loss[len(val_loss) - 1]

        return val_loss_value

    def get_validation_acc(self, hist):
        val_acc = hist.history['val_binary_accuracy']
        val_acc_value = val_acc[len(val_acc) - 1]

        return val_acc_value


    def print_metrics(self, hist):

        acc_value = self.get_binary_acc(hist)
        loss_value = self.get_binary_loss(hist)

        val_acc_value = self.get_validation_acc(hist)

        val_loss_value = self.get_validation_loss(hist)

        print("Final metrics: binary_loss:%6.4f" % loss_value)
        print("Final metrics: binary_accuracy=%6.4f" % acc_value)
        print("Final metrics: validation_binary_loss:%6.4f" % val_loss_value)
        print("Final metrics: validation_binary_accuracy:%6.4f" % val_acc_value)

    def get_directory_path(self, dir_name, create_dir=True):

        cwd = os.getcwd()

        dir = os.path.join(cwd, dir_name)
        if create_dir:
          if not os.path.exists(dir):
             os.mkdir(dir)

        return dir

    def train_models(self, args, base_line=True):
        """
        Train the model and log all the MLflow Metrics
        :param args: command line arguments. If no arguments then use default
        :param base_line: Default flag. Create Baseline model
        """
        with mlflow.start_run(experiment_id=11):
            # Create TensorFlow Session
            sess = tf.InteractiveSession()


            #
            # initialize some classes
            #
            kdata_cls = KIMDB_Data_Utils()
            ktrain_cls = KTrain()
            kplot_cls = KPlot()

            #
            # get IMDB Data
            #
            (train_data, train_labels), (test_data, test_labels) = kdata_cls.fetch_imdb_data()

            #
            # prepare and vectorize data
            #
            x_train = kdata_cls.prepare_vectorized_sequences(train_data)
            x_test = kdata_cls.prepare_vectorized_sequences(test_data)

            y_train = kdata_cls.prepare_vectorized_labels(train_labels)
            y_test = kdata_cls.prepare_vectorized_labels(test_labels)

            image_dir = ktrain_cls.get_directory_path("images")
            model_dir = ktrain_cls.get_directory_path("models")

            graph_label_loss = 'Baseline Model: Training and Validation Loss'
            graph_label_acc = 'Baseline Model: Training and Validation Accuracy'
            graph_image_loss_png = os.path.join(image_dir,'baseline_loss.png')
            graph_image_acc_png = os.path.join(image_dir, 'baseline_accuracy.png')

            if not base_line:
                graph_label_loss = 'Experimental: Training and Validation Loss'
                graph_label_acc = 'Experimental Model: Training and Validation Accuracy'
                graph_image_loss_png = os.path.join(image_dir, 'experimental_loss.png')
                graph_image_acc_png = os.path.join(image_dir,'experimental_accuracy.png')

            kmodel = KModel()
            if base_line:
                print("Baseline Model:")
                model = kmodel.build_basic_model()
            else:
                print("Experiment Model:")
                model = kmodel.build_experimental_model(args.hidden_layers, args.output)

            history = ktrain_cls.compile_and_fit_model(model, x_train, y_train, epochs=args.epochs, loss=args.loss)
            model.summary()
            ktrain_cls.print_metrics(history)

            figure_loss = kplot_cls.plot_loss_graph(history, graph_label_loss)
            figure_loss.savefig(graph_image_loss_png )

            figure_acc = kplot_cls.plot_accuracy_graph(history, graph_label_acc)
            figure_acc.savefig(graph_image_acc_png)

            results = ktrain_cls.evaulate_model(model, x_test, y_test)

            print("Average Probability Results:")
            print(results)

            print()
            print("Predictions Results:")
            predictions = model.predict(x_test)
            print(predictions)

            # print out current run_uuid
            run_uuid = mlflow.active_run().info.run_uuid
            print("MLflow Run ID: %s" % run_uuid)
            
            # log parameters
            mlflow.log_param("hidden_layers", args.hidden_layers)
            mlflow.log_param("output", args.output)
            mlflow.log_param("epochs", args.epochs)
            mlflow.log_param("loss_function", args.loss)
            
            # log metrics
            mlflow.log_metric("binary_loss", ktrain_cls.get_binary_loss(history))
            mlflow.log_metric("binary_acc",  ktrain_cls.get_binary_acc(history))
            mlflow.log_metric("validation_loss", ktrain_cls.get_binary_loss(history))
            mlflow.log_metric("validation_acc", ktrain_cls.get_validation_acc(history))
            mlflow.log_metric("average_loss", results[0])
            mlflow.log_metric("average_acc", results[1])
            
            # log artifacts
            mlflow.log_artifacts(image_dir, "images")
                        
            # log model
            mlflow.keras.log_model(model, "models")

            # save model locally
            pathdir = "keras_models/" + run_uuid
            model_dir = self.get_directory_path(pathdir, False)
            ktrain_cls.keras_save_model(model, model_dir)

            # Write out tensorflow graph
            output_dir = tempfile.mkdtemp()
            print("Writing TensorFlow events locally to %s\n" % output_dir)
            writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
            print("Uploading TensorFlow events as a run artifact.")
            mlflow.log_artifacts(output_dir, artifact_path="events")

        print("loss function use", args.loss)

        # Close TensorFlow Session
        sess.close()

  
def runReviews(args, flag):
    
    if flag:
        print("Using Default Baseline parameters")
    else:
        print("Using Experimental parameters")

    print("hidden_layers:", args.hidden_layers)
    print("output:", args.output)
    print("epochs:", args.epochs)
    print("loss:", args.loss)

    train_models_cls = KTrain().train_models(args, flag)

In [8]:
args = Params(1, 20, 16, "binary_crossentropy")
runReviews(args, True)

Using Default Baseline parameters
('hidden_layers:', 1)
('output:', 16)
('epochs:', 20)
('loss:', 'binary_crossentropy')
Baseline Model:
Train on 15000 samples, validate on 10000 samples
Epoch 1/20

 512/15000 [>.............................] - ETA: 8s - loss: 0.6939 - binary_accuracy: 0.5020
 1024/15000 [=>............................] - ETA: 5s - loss: 0.6930 - binary_accuracy: 0.5107
 1536/15000 [==>...........................] - ETA: 4s - loss: 0.6933 - binary_accuracy: 0.5046
 2048/15000 [===>..........................] - ETA: 4s - loss: 0.6935 - binary_accuracy: 0.4956
 2560/15000 [====>.........................] - ETA: 3s - loss: 0.6927 - binary_accuracy: 0.5043
 3072/15000 [=====>........................] - ETA: 3s - loss: 0.6928 - binary_accuracy: 0.5085
 3584/15000 [======>.......................] - ETA: 3s - loss: 0.6926 - binary_accuracy: 0.5112
 4096/15000 [=======>......................] - ETA: 2s - loss: 0.6923 - binary_accuracy: 0.5134
 4608/15000 [========>.....................] - ETA: 2s - loss: 0.6919 - binary_accuracy: 0.5197
 5120/15000 [=========>....................] - ETA: 2s - loss: 0.6915 - binary_accuracy: 0.5281
 5632/15000 [==========>...................] - ETA: 2s - loss: 0.6912 - binary_accuracy: 0.5293
 6144/15000 [===========>..................] - ETA: 2s - loss: 0.6911 - binary_accuracy: 0.5308
 6656/15000 [============>.................] - ETA: 2s - loss: 0.6907 - binary_accuracy: 0.5328
 7168/15000 [=============>................] - ETA: 1s - loss: 0.6902 - binary_accuracy: 0.5340
 7680/15000 [==============>...............] - ETA: 1s - loss: 0.6895 - binary_accuracy: 0.5401
 8192/15000 [===============>..............] - ETA: 1s - loss: 0.6889 - binary_accuracy: 0.5446
 8704/15000 [================>.............] - ETA: 1s - loss: 0.6886 - binary_accuracy: 0.5465
 9216/15000 [=================>............] - ETA: 1s - loss: 0.6883 - binary_accuracy: 0.5492
 9728/15000 [==================>...........] - ETA: 1s - loss: 0.6880 - binary_accuracy: 0.5514
10240/15000 [===================>..........] - ETA: 1s - loss: 0.6875 - binary_accuracy: 0.5552
10752/15000 [====================>.........] - ETA: 0s - loss: 0.6873 - binary_accuracy: 0.5573
11264/15000 [=====================>........] - ETA: 0s - loss: 0.6866 - binary_accuracy: 0.5611
11776/15000 [======================>.......] - ETA: 0s - loss: 0.6859 - binary_accuracy: 0.5639
12288/15000 [=======================>......] - ETA: 0s - loss: 0.6853 - binary_accuracy: 0.5671
12800/15000 [========================>.....] - ETA: 0s - loss: 0.6847 - binary_accuracy: 0.5695
13312/15000 [=========================>....] - ETA: 0s - loss: 0.6841 - binary_accuracy: 0.5720
13824/15000 [==========================>...] - ETA: 0s - loss: 0.6837 - binary_accuracy: 0.5734
14336/15000 [===========================>..] - ETA: 0s - loss: 0.6832 - binary_accuracy: 0.5756
14848/15000 [============================>.] - ETA: 0s - loss: 0.6828 - binary_accuracy: 0.5768
15000/15000 [==============================] - 5s 345us/step - loss: 0.6826 - binary_accuracy: 0.5781 - val_loss: 0.6665 - val_binary_accuracy: 0.6507
Epoch 2/20

 512/15000 [>.............................] - ETA: 3s - loss: 0.6602 - binary_accuracy: 0.6953
 1024/15000 [=>............................] - ETA: 3s - loss: 0.6593 - binary_accuracy: 0.6875
 1536/15000 [==>...........................] - ETA: 3s - loss: 0.6586 - binary_accuracy: 0.6895
 2048/15000 [===>..........................] - ETA: 2s - loss: 0.6582 - binary_accuracy: 0.6904
 2560/15000 [====>.........................] - ETA: 2s - loss: 0.6582 - binary_accuracy: 0.6887
 3072/15000 [=====>........................] - ETA: 2s - loss: 0.6573 - binary_accuracy: 0.6895
 3584/15000 [======>.......................] - ETA: 2s - loss: 0.6577 - binary_accuracy: 0.6855
 4096/15000 [=======>......................] - ETA: 2s - loss: 0.6568 - binary_accuracy: 0.6919
 4608/15000 [========>.....................] - ETA: 2s - loss: 0.6556 - binary_accuracy:

In [9]:
args = Params(3, 30, 16, "binary_crossentropy")
runReviews(args, False)

Using Experimental parameters
('hidden_layers:', 3)
('output:', 16)
('epochs:', 30)
('loss:', 'binary_crossentropy')
Experiment Model:
Train on 15000 samples, validate on 10000 samples
Epoch 1/30

 512/15000 [>.............................] - ETA: 10s - loss: 0.6937 - binary_accuracy: 0.5078
 1024/15000 [=>............................] - ETA: 6s - loss: 0.6937 - binary_accuracy: 0.5137 
 1536/15000 [==>...........................] - ETA: 5s - loss: 0.6933 - binary_accuracy: 0.5078
 2048/15000 [===>..........................] - ETA: 4s - loss: 0.6933 - binary_accuracy: 0.5024
 2560/15000 [====>.........................] - ETA: 4s - loss: 0.6932 - binary_accuracy: 0.5000
 3072/15000 [=====>........................] - ETA: 3s - loss: 0.6932 - binary_accuracy: 0.4964
 3584/15000 [======>.......................] - ETA: 3s - loss: 0.6931 - binary_accuracy: 0.4978
 4096/15000 [=======>......................] - ETA: 3s - loss: 0.6929 - binary_accuracy: 0.4971
 4608/15000 [========>.....................] - ETA: 2s - loss: 0.6927 - binary_accuracy: 0.4998
 5120/15000 [=========>....................] - ETA: 2s - loss: 0.6925 - binary_accuracy: 0.5004
 5632/15000 [==========>...................] - ETA: 2s - loss: 0.6924 - binary_accuracy: 0.5011
 6144/15000 [===========>..................] - ETA: 2s - loss: 0.6922 - binary_accuracy: 0.5010
 6656/15000 [============>.................] - ETA: 2s - loss: 0.6920 - binary_accuracy: 0.5024
 7168/15000 [=============>................] - ETA: 2s - loss: 0.6920 - binary_accuracy: 0.5008
 7680/15000 [==============>...............] - ETA: 1s - loss: 0.6920 - binary_accuracy: 0.4982
 8192/15000 [===============>..............] - ETA: 1s - loss: 0.6918 - binary_accuracy: 0.5010
 8704/15000 [================>.............] - ETA: 1s - loss: 0.6917 - binary_accuracy: 0.5009
 9216/15000 [=================>............] - ETA: 1s - loss: 0.6914 - binary_accuracy: 0.5015
 9728/15000 [==================>...........] - ETA: 1s - loss: 0.6913 - binary_accuracy: 0.5009
10240/15000 [===================>..........] - ETA: 1s - loss: 0.6912 - binary_accuracy: 0.4997
10752/15000 [====================>.........] - ETA: 1s - loss: 0.6912 - binary_accuracy: 0.4985
11264/15000 [=====================>........] - ETA: 0s - loss: 0.6912 - binary_accuracy: 0.4971
11776/15000 [======================>.......] - ETA: 0s - loss: 0.6911 - binary_accuracy: 0.4979
12288/15000 [=======================>......] - ETA: 0s - loss: 0.6910 - binary_accuracy: 0.4967
12800/15000 [========================>.....] - ETA: 0s - loss: 0.6909 - binary_accuracy: 0.4964
13312/15000 [=========================>....] - ETA: 0s - loss: 0.6909 - binary_accuracy: 0.4956
13824/15000 [==========================>...] - ETA: 0s - loss: 0.6907 - binary_accuracy: 0.4966
14336/15000 [===========================>..] - ETA: 0s - loss: 0.6904 - binary_accuracy: 0.4971
14848/15000 [============================>.] - ETA: 0s - loss: 0.6902 - binary_accuracy: 0.4985
15000/15000 [==============================] - 6s 381us/step - loss: 0.6902 - binary_accuracy: 0.4981 - val_loss: 0.6844 - val_binary_accuracy: 0.5108
Epoch 2/30

 512/15000 [>.............................] - ETA: 3s - loss: 0.6803 - binary_accuracy: 0.5508
 1024/15000 [=>............................] - ETA: 3s - loss: 0.6807 - binary_accuracy: 0.5420
 1536/15000 [==>...........................] - ETA: 3s - loss: 0.6820 - binary_accuracy: 0.5208
 2048/15000 [===>..........................] - ETA: 2s - loss: 0.6821 - binary_accuracy: 0.5142
 2560/15000 [====>.........................] - ETA: 2s - loss: 0.6821 - binary_accuracy: 0.5094
 3072/15000 [=====>........................] - ETA: 2s - loss: 0.6823 - binary_accuracy: 0.5055
 3584/15000 [======>.......................] - ETA: 2s - loss: 0.6818 - binary_accuracy: 0.5095
 4096/15000 [=======>......................] - ETA: 2s - loss: 0.6816 - binary_accuracy: 0.5083
 4608/15000 [========>.....................] - ETA: 2s - loss: 0.6813 - binary_accuracy:

In [10]:
args = Params(3, 30, 32, "mse")
runReviews(args, True)

Using Default Baseline parameters
('hidden_layers:', 3)
('output:', 32)
('epochs:', 30)
('loss:', 'mse')
Baseline Model:
Train on 15000 samples, validate on 10000 samples
Epoch 1/30

 512/15000 [>.............................] - ETA: 11s - loss: 0.2504 - binary_accuracy: 0.5020
 1024/15000 [=>............................] - ETA: 7s - loss: 0.2499 - binary_accuracy: 0.5107 
 1536/15000 [==>...........................] - ETA: 5s - loss: 0.2501 - binary_accuracy: 0.5046
 2048/15000 [===>..........................] - ETA: 4s - loss: 0.2502 - binary_accuracy: 0.4956
 2560/15000 [====>.........................] - ETA: 3s - loss: 0.2498 - binary_accuracy: 0.5043
 3072/15000 [=====>........................] - ETA: 3s - loss: 0.2498 - binary_accuracy: 0.5078
 3584/15000 [======>.......................] - ETA: 3s - loss: 0.2497 - binary_accuracy: 0.5103
 4096/15000 [=======>......................] - ETA: 3s - loss: 0.2496 - binary_accuracy: 0.5125
 4608/15000 [========>.....................] - ETA: 2s - loss: 0.2494 - binary_accuracy: 0.5187
 5120/15000 [=========>....................] - ETA: 2s - loss: 0.2492 - binary_accuracy: 0.5271
 5632/15000 [==========>...................] - ETA: 2s - loss: 0.2490 - binary_accuracy: 0.5291
 6144/15000 [===========>..................] - ETA: 2s - loss: 0.2490 - binary_accuracy: 0.5308
 6656/15000 [============>.................] - ETA: 2s - loss: 0.2488 - binary_accuracy: 0.5329
 7168/15000 [=============>................] - ETA: 1s - loss: 0.2485 - binary_accuracy: 0.5346
 7680/15000 [==============>...............] - ETA: 1s - loss: 0.2482 - binary_accuracy: 0.5406
 8192/15000 [===============>..............] - ETA: 1s - loss: 0.2479 - binary_accuracy: 0.5450
 8704/15000 [================>.............] - ETA: 1s - loss: 0.2477 - binary_accuracy: 0.5470
 9216/15000 [=================>............] - ETA: 1s - loss: 0.2476 - binary_accuracy: 0.5497
 9728/15000 [==================>...........] - ETA: 1s - loss: 0.2474 - binary_accuracy: 0.5522
10240/15000 [===================>..........] - ETA: 1s - loss: 0.2472 - binary_accuracy: 0.5558
10752/15000 [====================>.........] - ETA: 0s - loss: 0.2471 - binary_accuracy: 0.5579
11264/15000 [=====================>........] - ETA: 0s - loss: 0.2467 - binary_accuracy: 0.5616
11776/15000 [======================>.......] - ETA: 0s - loss: 0.2464 - binary_accuracy: 0.5643
12288/15000 [=======================>......] - ETA: 0s - loss: 0.2461 - binary_accuracy: 0.5675
12800/15000 [========================>.....] - ETA: 0s - loss: 0.2458 - binary_accuracy: 0.5700
13312/15000 [=========================>....] - ETA: 0s - loss: 0.2455 - binary_accuracy: 0.5724
13824/15000 [==========================>...] - ETA: 0s - loss: 0.2453 - binary_accuracy: 0.5737
14336/15000 [===========================>..] - ETA: 0s - loss: 0.2451 - binary_accuracy: 0.5760
14848/15000 [============================>.] - ETA: 0s - loss: 0.2449 - binary_accuracy: 0.5773
15000/15000 [==============================] - 5s 338us/step - loss: 0.2448 - binary_accuracy: 0.5785 - val_loss: 0.2369 - val_binary_accuracy: 0.6499
Epoch 2/30

 512/15000 [>.............................] - ETA: 2s - loss: 0.2338 - binary_accuracy: 0.6934
 1024/15000 [=>............................] - ETA: 2s - loss: 0.2334 - binary_accuracy: 0.6855
 1536/15000 [==>...........................] - ETA: 2s - loss: 0.2330 - binary_accuracy: 0.6875
 2048/15000 [===>..........................] - ETA: 2s - loss: 0.2328 - binary_accuracy: 0.6899
 2560/15000 [====>.........................] - ETA: 2s - loss: 0.2328 - binary_accuracy: 0.6883
 3072/15000 [=====>........................] - ETA: 2s - loss: 0.2323 - binary_accuracy: 0.6898
 3584/15000 [======>.......................] - ETA: 2s - loss: 0.2325 - binary_accuracy: 0.6858
 4096/15000 [=======>......................] - ETA: 2s - loss: 0.2321 - binary_accuracy: 0.6926
 4608/15000 [========>.....................] - ETA: 2s - loss: 0.2315 - binary_accuracy: 0.6940
 5120/

In [11]:
# Update this manually by reviewing the previous cell `Writing TensorFlow events locally to...`
output_dir = "/tmp/tmp2npaqjhu"
dbutils.tensorboard.start(output_dir)

In [12]:
dbutils.tensorboard.stop()